In [17]:
# parameters
c = 64
s = 4
numleaf = 64
mult = 12
numerator = 15
denominator = 16
seed = 0
numserver = 3072
numsw = 80
numport = 64

In [18]:
# read leafspine serverfile
serverfile = f"/home/annzhou/DRing/src/emp/datacentre/serverfiles/leafspine_{numserver}_{numsw}_{numport}"
serverdict = dict()
with open(serverfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        serverdict[int(tokens[0])] = int(tokens[1])

In [19]:
import numpy as np
import random
random.seed(seed)

# inspired by int ConnectionMatrix::genFlowBytes()
def genflowbytes():
    mean_bytes = 100.0 * 1024
    shape = 1.05
    scale = mean_bytes * (shape - 1)/shape

    x = np.random.exponential(scale=1.0/shape)
    flowbytes = int(scale * np.exp(x))
    return flowbytes

def adjustbytesbymtu(flowbytes):
  mss = 1500
  return mss * ((flowbytes+mss-1)//mss)


allfromswitches = list()
alltoswitches = list()
if c==numleaf:
  allfromswitches = range(numleaf)
else:
  while len(allfromswitches)<c:
    rsw = random.randint(0,numleaf-1)
    if rsw not in allfromswitches:
      allfromswitches.append(rsw)
if s==numleaf:
  alltoswitches = range(numleaf)
else:
  while len(alltoswitches)<s:
    rsw = random.randint(0,numleaf-1)
    if rsw not in alltoswitches:
      alltoswitches.append(rsw)

allfromservers = list()
alltoservers = list()
for lsserver in range(numserver):
  lsswitch = serverdict[lsserver]
  if lsswitch in allfromswitches:
    allfromservers.append(lsserver)
  if lsswitch in alltoswitches:
    alltoservers.append(lsserver)

large_flow_threshold = 10 * 1024 * 1024
base_flows = list()
for fromsvr in allfromservers:
  for tosvr in alltoservers:
    if fromsvr!=tosvr:
      mybytes = genflowbytes()
      while mybytes<0 or mybytes>large_flow_threshold:
        mybytes = genflowbytes()
      mybytes = adjustbytesbymtu(mybytes)
      base_flows.append([fromsvr,tosvr,mybytes])

flowfile = f"/home/annzhou/DRing/src/emp/datacentre/flowfiles/c2s_{c}_{s}_{mult}_{numerator}_{denominator}_{seed}"
with open(flowfile,'w') as f:
  if mult>0:
    for m in range(mult):
      for flow in base_flows:
        f.write(f"{flow[0]},{flow[1]},{flow[2]}\n")

  if denominator>0:
    for flow in base_flows:
      r = random.randint(0,denominator-1) # inclusive
      if r<numerator:
        f.write(f"{flow[0]},{flow[1]},{flow[2]}\n")

In [20]:
# import numpy as np

# # chatgpt translation:
# # int genflowbytes(double shape, double scale) {
# #     exp_distribution = std::exponential_distribution<double>(shape);
# #     double x = exp_distribution(generator);
# #     int bytes = scale * exp(x);
# #     return bytes;
# # }

# # def genflowbytes(shape, scale):
# #     # Generate a random sample from an exponential distribution
# #     x = np.random.exponential(scale=shape) <=== AnnC: wrong, scale=1.0/shape
    
# #     # Compute the result using the exponential function
# #     bytes = int(scale * np.exp(x))
    
# #     return bytes

# mean_bytes = 100.0 * 1024
# shape = 1.05
# myscale = mean_bytes * (shape - 1)/shape

# def genflowbytes():
#     x = np.random.exponential(scale=1.0/shape)
#     flowbytes = int(myscale * np.exp(x))
#     return flowbytes

# large_flow_threshold = 10 * 1024 * 1024
# print(f"mean={mean_bytes},shape={shape},scale={myscale},threshold={large_flow_threshold}")
# sumbytes = 0
# countbytes = 500000
# for i in range(countbytes):
#     mybytes = genflowbytes()
#     while mybytes<0 or mybytes>large_flow_threshold:
#         mybytes = genflowbytes()
#     sumbytes += mybytes
# print(sumbytes/countbytes)